In [ ]:
import os
import json
import yaml

from IPython.display import display, Markdown

import numpy as np
import pandas as pd

# from decode_trf import delivery_data_from_logfile

from pymedphys.msq import mosaiq_connect
from pymedphys.utilities import get_index, get_data_directory, get_filepath, get_gantry_tolerance
from pymedphys.logfile import *

from pymedphys.plt import pcolormesh_grid

In [ ]:
with open('../config.json') as config_file:
    config = json.load(config_file)

In [ ]:
index = get_index(config)

In [ ]:
data_directory = get_data_directory(config)

cache_filepath = os.path.join(data_directory, 'cache', 'dmlc_comparison.yaml')
cache_scratch_filepath = os.path.join(data_directory, 'cache', 'dmlc_comparison_scratch.yaml')

with open(cache_filepath, 'r') as cache_file:
    cache = yaml.load(cache_file)

In [ ]:
patient_ids = list(cache.keys())
len(patient_ids)

In [ ]:
data = []

for patient_id in patient_ids:
    for byfield in cache[patient_id]:
        for angle in cache[patient_id][byfield]:
#             print(cache[patient_id][byfield][angle].keys())
            comparison = cache[patient_id][byfield][angle]['median']
            file_hashes = cache[patient_id][byfield][angle]['median_filehash_group']
            all_comparisons = cache[patient_id][byfield][angle]['comparisons']
            all_file_hashes = cache[patient_id][byfield][angle]['filehash_groups']
            all_comparisons_flat = []
            for key, value in all_comparisons.items():
                all_comparisons_flat.append(value)
            
            data.append([
                patient_id, byfield, angle, comparison, file_hashes, tuple(all_comparisons_flat), tuple(all_file_hashes)
            ])
            
comparisons_table = pd.DataFrame(
    columns=[
        'patient_id', 'field_id',
        'gantry_angle', 'comparison',
        'file_hashes', 'all_comparisons',
        'all_file_hashes'
    ],
    data=data
)

comparisons_table = comparisons_table.sort_values('comparison', ascending=False)
top_ten = comparisons_table.iloc[0:10]
top_ten

In [ ]:
field_id = 77630

In [ ]:
field_ref = comparisons_table['field_id'] == field_id
comparisons_table[field_ref]

In [ ]:
worst_row_of_field = comparisons_table[field_ref].iloc[0]

In [ ]:
gantry_angle = worst_row_of_field['gantry_angle']
gantry_angle

In [ ]:
gantry_angles = comparisons_table[field_ref]['gantry_angle'].values
gantry_angles

In [ ]:
worst_row_of_field['all_file_hashes']

In [ ]:
worst_file_ref = np.argmax(worst_row_of_field['all_comparisons'])
logfile_group = worst_row_of_field['all_file_hashes'][worst_file_ref]
logfile_group

In [ ]:
file_hash = logfile_group[0]
file_hash

In [ ]:
# with mosaiq_connect('msqsql') as cursor:
#     comparison = compare_logfile_group_bygantry(
#         index, config, cursor, logfile_group, gantry_angle)

In [ ]:
filepath = get_filepath(index, config, file_hash)
filepath

In [ ]:
with mosaiq_connect('msqsql') as cursor:
    mosaiq_delivery_data = multi_fetch_and_verify_mosaiq(cursor, field_id)

In [ ]:
grid_resolution = 0.25

In [ ]:
mosaiq_mu_density = calc_mu_density_bygantry(
    mosaiq_delivery_data, gantry_angle, grid_resolution)

normalisation = calc_normalisation(mosaiq_delivery_data)

logfile_mu_density = calc_logfile_mu_density_bygantry(
    index, config, logfile_group, gantry_angle, grid_resolution)

grid_xx = logfile_mu_density[0]
grid_yy = logfile_mu_density[1]

logfile_mu_density = logfile_mu_density[2]
mosaiq_mu_density = mosaiq_mu_density[2]

In [ ]:
min_val = np.min([logfile_mu_density, mosaiq_mu_density])
max_val = np.max([logfile_mu_density, mosaiq_mu_density])

In [ ]:
x = grid_xx[0,:]
y = grid_yy[:,0]

x, y = pcolormesh_grid(x, y)

x = -x

In [ ]:
x

In [ ]:
import json

In [ ]:
np.round(logfile_mu_density, 2).tolist()

In [ ]:
to_save = {
    'x': x.tolist(),
    'y': y.tolist(),
    'logfile': np.round(logfile_mu_density, 2).tolist(),
    'mosaiq': np.round(mosaiq_mu_density, 2).tolist()
}

In [ ]:
save_file_path = r'S:\Physics\Programming\data\LinacLogFiles\results\EPSM2018_77630_investigation.json'

In [ ]:
with open(save_file_path, 'w') as save_file:
    json.dump(to_save, save_file)

In [ ]:
plt.figure(figsize=(10,7))
plt.pcolormesh(x, y, logfile_mu_density, vmin=min_val, vmax=max_val)
plt.colorbar(label='hello')
plt.title('Logfile MU density')
plt.xlabel('MLC direction (mm)')
plt.ylabel('Jaw direction (mm)')

plt.axis('equal')

plt.xlim([-67, 60])
plt.ylim([60, -75])

In [ ]:
get_logfile_delivery_data_bygantry(
    index, config, [logfile_group], gantry_angles)